# Setup

In [366]:
%pip install openai tiktoken

Note: you may need to restart the kernel to use updated packages.


In [367]:
from openai import OpenAI
client = OpenAI()
llm = "gpt-4-1106-preview"

## Choose an example

In [368]:
job_options=['medical-assistant', 'infra-engineer', 'customer-success-manager', 'real-estate-litigator','growth-manager','product-designer']

choice = job_options[1]

job_description_file = "examples/" + choice + "/job-description.md"
company_file = "examples/" + choice + "/about-company.md"

# destination files
destination_directory = "examples/" + choice + "/"
draft_destination = destination_directory + choice + '-draft.md'
word_draft_destination = destination_directory + choice + '-draft.docx'
markdown_destination = destination_directory + choice + '-final.md'
word_destination = destination_directory + choice + '-final.docx'



In [369]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        # print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        # print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


def num_tokens_from_string(string, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a string."""
    return num_tokens_from_messages([{"content": string}], model=model)


# Load the Assistant

## System Prompt

In [370]:
from datetime import datetime


# basic instructions
system_prompt = """
You are a Talent Acquisition and Development Specialist. Your task is to do a job analysis and 
identify the knowledge, skills, abilities, and other characteristics (KSAOs) as 5-10 competencies 
for a hiring team to assess in a candidate.  At least 4 competencies should cover the technical 
skills of the job.
Lower level jobs should have less competencies and higher level jobs should have more, but you 
should cover all the requirements in the job desccription.  You should also include competencies 
or word competencies that are aligned with the company values.
Cite and quote the job description and company values as appropriate.
Include the job description verbatim, but change the heading levels to be just bolded text and include a blank line before lists.
Be sure to cover specific technologies, systems or skills mentioned in the job description.

Tone: spartan, use less corporate jargon. Write like Hemingway.
Don't use the words 'technological', 'tranformational', 'foster', 'fostering', 'tapestry', 'like', 'overall', 'furthermore'
Don't use the phrases 'this is about', 'all about', 'think of', 'not only', 'but also', 'in conclusion'
Refer to the actual company name rather than 'organization' or 'company'.
Vary your sentence length and structure.
Avoid repeating the same word in a sentence or paragraph.

If you are asked to review the document:
- follow the Jinja2 template exactly, including executing any instructions or loops in the template
- if any sections from the template have been truncated or omited, please complete them now
- improve the quality and accuracy of the writing throughout the document (except for the job description)
- review the draft competencies and add, delete or modify as appropriate
- review the competency detail sections, interview questions, case study, and candidate Q&A to improve them as appropriate
- if appropriate, add a bonus competency that is not mentioned in the job description but you feel is very important for the role. describe why you have added it.
- be sure that the technologies, systems or skills mentioned in the job description are covered in the guide
- discuss your improvements in the <thinking></thinking> tags
- do not modify the job description section as this has been provided to you by the hiring manager
- cite and or quote the job description as appropriate
- don't include any delimiters or commentary in the output

Make sure that you always follow the template from your system instructions EXACTLY and completely.

"""

# system_prompt += "\n\nFor this test, only produce 1 competencies."

# add today's date and time to include in the document
system_prompt += "\nToday's date and time is " + datetime.now().strftime("%B %d, %Y %-I:%M %p") + ".\n"

# instructions for processing the document template. We will reuse this.
doc_prompt = """
You will be provided below with a template in Jinja2 syntax delimited by triple quotes, and your task is to fully and accurately populate the template.
Always complete all loops fully. 
Always complete the metadata.
ALWAYS include a blank line after a markdown block and before a list.
Do NOT omit or truncate any content or sections. 
Do NOT omit blank spaces or lines.
Do NOT include any delimiters, backticks or commentary in the output.
Where prompted, provide your thinking about what you will write and include your thinking in <thinking></thinking> tags.

When you are entirely finished with the entire document, write STOPSTOP.

The Jinja2 template that you will follow faithfully is as follows:
"""
system_prompt += doc_prompt

# add the document template
with open('templates/prompt_template.md', 'r') as file:
    document_template = file.read()

system_prompt += "\n\"\"\"\n" + document_template + "\n\"\"\"\n\n"
    
print(system_prompt)


You are a Talent Acquisition and Development Specialist. Your task is to do a job analysis and 
identify the knowledge, skills, abilities, and other characteristics (KSAOs) as 5-10 competencies 
for a hiring team to assess in a candidate.  At least 4 competencies should cover the technical 
skills of the job.
Lower level jobs should have less competencies and higher level jobs should have more, but you 
should cover all the requirements in the job desccription.  You should also include competencies 
or word competencies that are aligned with the company values.
Cite and quote the job description and company values as appropriate.
Include the job description verbatim, but change the heading levels to be just bolded text and include a blank line before lists.
Be sure to cover specific technologies, systems or skills mentioned in the job description.

Tone: spartan, use less corporate jargon. Write like Hemingway.
Don't use the word 'technological', 'tranformational', 'foster', 'fosterin

## Job Description

In [371]:
def get_job_description():
    job_description = ""
    with open(job_description_file, "r") as file:
        job_description = file.read()
    return job_description

def get_company_description():
    company_description = ""
    with open(company_file, "r") as file:
        company_description = file.read()
    return company_description

def build_user_prompt(job_description, company_description):
    user_prompt = "I am supplying you with a job description and a company description.\n\n"
    user_prompt += "Job description is as follows:\n\n\"\"\"\n" + job_description + "\n\"\"\"\n\n"
    user_prompt += "Company description is as follows:\n\n\"\"\"\n" + company_description + "\n\"\"\"\n\n"
    user_prompt += "Proceed generating the full document. Do not truncate or abridge the document. You MUST follow the template EXACTLY.\n\n"
    return user_prompt

job_description = get_job_description()
company_description = get_company_description()

user_prompt = build_user_prompt(job_description, company_description)

print(user_prompt)


I am supplying you with a job description and a company description.

Job description is as follows:

"""
    ## Job Description

    **Y Combinator - Infrastructure Software Engineer**

    Y Combinator is seeking an Infrastructure Software Engineer who will help manage and evolve the shared infrastructure that our software runs on. We have a team of approximately 15 Product Engineers who develop YC’s software products. Those products are developed in Ruby on Rails and run in AWS. Our team’s job is to provide a platform so our Product Engineers don’t have to think (as much) about infrastructure.

    The ideal candidate will have experience debugging and developing a very broad stack. They’re comfortable talking about HTTP caching, AWS security groups, Rails monkey patching, n+1 queries in Active Record, CSRF protection, and React components.

    Enjoy the many perks of working for a successful company – competitive salary and excellent benefits including fully-paid health care benef

## Product the Draft

In [372]:
import re

# function to remove thinking tags
def remove_thinking_tags(string):
    pattern = r"<thinking>.*?</thinking>"
    return re.sub(pattern, "", string)

def insert_blank_lines(string):
    pattern = r"(?<=\n)(?=[*-]\s)"
    return re.sub(pattern, "\n", string)


In [373]:
# function to generate the output from GPT-4
def generate_output(system_prompt, user_prompt, llm):
    STOP_SIGNAL = "STOPSTOP"
    output = "" 

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    input_tokens = 0
    output_tokens = 0

    while True:
        try:
            stream = client.chat.completions.create(
                model=llm,
                max_tokens=4096,
                temperature=0,
                messages=messages,
                stream=True,
                stop=[STOP_SIGNAL],
                frequency_penalty=0,
                presence_penalty=0
            )
            input_tokens += num_tokens_from_messages(messages, model=llm)

            new_output = ""

            for chunk in stream:
                if chunk.choices[0].delta.content is not None:
                    print(chunk.choices[0].delta.content, end="")
                    new_chunk = chunk.choices[0].delta.content
                    new_output += new_chunk

                if STOP_SIGNAL in new_output:
                    break

                if chunk.choices[0].delta.content is None:
                    break

            output_tokens += num_tokens_from_string(new_output, model=llm)       
            output += new_output
            if not new_output or STOP_SIGNAL in new_output:
                break

            messages.append({"role": "assistant", "content": new_output})
        except Exception as e:
            print("Error:", e)
            # Handle the error here, such as retrying or terminating the loop
    
    output = output.replace(STOP_SIGNAL, "") # remove the stop signal from the output
    output = output.replace("```", "") # remove backticks from the output if they have snuck through
    output = remove_thinking_tags(output) # remove the <thinking></thinking> tags from the output
    output = insert_blank_lines(output) # insert blank lines before lists

    return output, input_tokens, output_tokens



In [374]:
# generate the output from the LLM
draft_markdown, input_tokens, output_tokens = generate_output(system_prompt, user_prompt, llm) 

---
title: Interview Guide for Infrastructure Software Engineer
subtitle: Y Combinator
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 24, 2024 8:19 PM
--- 

# Interview Guide for Infrastructure Software Engineer

<thinking>When creating this guide, I'm considering the specific requirements and responsibilities outlined in the job description for the Infrastructure Software Engineer role at Y Combinator. The guide will be tailored to assess the candidate's technical skills, experience, and alignment with Y Combinator's values.</thinking>

## About this Guide

<thinking>The introduction should set the stage for the interview process, explaining its purpose, structure, and how it aligns with the role and Y Combinator's values.</thinking>

This guide is designed to streamline the interview process for the Infrastructure Software Engineer position at Y Combinator. It provides a structured approach to evaluate candidates' technical abilities, probl

In [375]:
# Save the final_markdown to the file
with open(draft_destination, "w") as file:
    file.write(draft_markdown)

# Print the file path
print("Draft markdown saved to:", draft_destination)

# Convert from markdown to docx
!pandoc -s {draft_destination} -o {word_draft_destination} --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=1 --shift-heading-level-by -1

# Print the file path
print("Draft Word doc saved to:", word_draft_destination)

Draft markdown saved to: examples/infra-engineer/infra-engineer-draft.md
Draft Word doc saved to: examples/infra-engineer/infra-engineer-draft.docx


In [376]:
# give update on the cost

cost = (input_tokens / 1000 * 0.01) + (output_tokens / 1000 * 0.03)
rounded_cost = round(cost, 2)
print("Total cost is approximately $" + str(rounded_cost) + " USD.")

Total cost is approximately $0.33 USD.


# Produce the Final Outputs

In [377]:
user_prompt = f"""
You are a Talent Acquisition and Development Specialist. Your task is to review a draft hiring guide that is in draft. 
Please review and improve it as necessary. 


Job description is as follows:
\"\"\"
{job_description}
\"\"\"

Company description is as follows:
\"\"\"
{company_description}
\"\"\"

Draft interview guide is as follows:
\"\"\"
{draft_markdown}
\"\"\"

The template is as follows:
\"\"\"
{document_template}
\"\"\"

"""
output, input_tokens, output_tokens = generate_output(system_prompt, user_prompt, llm)
final_markdown = output.replace("STOPSTOP", "")

# Define the file path
file_path = markdown_destination

# Save the final_markdown to the file
with open(file_path, "w") as file:
    file.write(final_markdown)

# Print the file path
print("Final markdown saved to:", file_path)



--- 
title: Interview Guide for Infrastructure Software Engineer
subtitle: Y Combinator
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 24, 2024 8:19 PM
--- 

# Interview Guide for Infrastructure Software Engineer

<thinking>Considering the role's technical nature, the guide should be straightforward and focused on practical skills, with a nod to Y Combinator's values. It should help interviewers assess both technical prowess and cultural fit.</thinking>

## About this Guide

<thinking>The introduction should set the stage for the interview process, emphasizing the importance of the role and how the guide will assist in finding the right candidate.</thinking>

This guide is a tool for interviewing candidates for the Infrastructure Software Engineer role at Y Combinator. It is crafted to assess the technical skills required to manage and evolve our software infrastructure, as well as the personal attributes that align with our values. 

The gui

In [378]:
# update on the cost
cost = (input_tokens / 1000 * 0.01) + (output_tokens / 1000 * 0.03)
rounded_cost = round(cost, 2)
print("Total cost is approximately $" + str(rounded_cost) + " USD.")

Total cost is approximately $0.49 USD.


# Final Output

In [379]:
# Convert from markdown to docx
!pandoc -s {markdown_destination} -o {word_destination} --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=1 --shift-heading-level-by -1

In [380]:

# display the final_markdown in the notebook
from IPython.display import Markdown
Markdown(final_markdown)

--- 
title: Interview Guide for Infrastructure Software Engineer
subtitle: Y Combinator
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 24, 2024 8:19 PM
--- 

# Interview Guide for Infrastructure Software Engineer



## About this Guide



This guide is a tool for interviewing candidates for the Infrastructure Software Engineer role at Y Combinator. It is crafted to assess the technical skills required to manage and evolve our software infrastructure, as well as the personal attributes that align with our values. 

The guide includes an interview agenda, a competency framework, and a case study. The agenda provides a structured flow for the interview. The competency framework outlines the skills and attributes to assess. The case study presents a problem for candidates to solve, showcasing their practical abilities.

We have integrated Y Combinator's values into the interview process. Candidates should demonstrate integrity, respect, accountability, and a founder-first mindset. This guide will help interviewers identify candidates who are not only technically capable but also a cultural fit for Y Combinator.

### Interview Agenda



1. Introduction and Icebreaker (10 minutes)
2. Review of Candidate's Background and Experience (20 minutes)
3. Technical Skills Assessment (30 minutes)
4. Competency-Based Questions and Scenarios (40 minutes)
5. Case Study Presentation and Discussion (30 minutes)
6. Candidate's Questions and Wrap-Up (10 minutes)

### Competency Framework



The competency framework for the Infrastructure Software Engineer role is designed to capture the essence of what makes a successful candidate. It is based on the job description and the values of Y Combinator. The framework includes technical competencies that are critical for the role, as well as interpersonal skills that align with our culture.

1. **Cloud Infrastructure Management**: Expertise in AWS services, including ECS, RDS, IAM, and CloudWatch.
2. **Web Application Development**: Proficiency in MVC web applications, with a focus on Ruby on Rails, Postgres, and React.
3. **Deployment and Operations**: Experience with AWS, Docker, and infrastructure as code using Terraform.
4. **Security and Performance**: Ability to secure applications and data, and debug performance issues.
5. **Collaboration and Communication**: Strong communication skills and the ability to work effectively with teams.
6. **Problem-Solving and Agility**: Demonstrated problem-solving skills and adaptability to change.

## Job Description

**Y Combinator - Infrastructure Software Engineer**

Y Combinator is seeking an Infrastructure Software Engineer who will help manage and evolve the shared infrastructure that our software runs on. We have a team of approximately 15 Product Engineers who develop YC’s software products. Those products are developed in Ruby on Rails and run in AWS. Our team’s job is to provide a platform so our Product Engineers don’t have to think (as much) about infrastructure.

The ideal candidate will have experience debugging and developing a very broad stack. They’re comfortable talking about HTTP caching, AWS security groups, Rails monkey patching, n+1 queries in Active Record, CSRF protection, and React components.

Enjoy the many perks of working for a successful company – competitive salary and excellent benefits including fully-paid health care benefits and unlimited vacation – while working on a small, fast-moving and high-impact team.

In your first month, you might:


- Migrate a Rails application from Capistrano and EC2 to Docker and ECS

- Upgrade one of our applications to a newer version of Postgres or Rails

- Move more of our AWS infrastructure into Terraform

- Simplify our use of Webpack and Javascript for both developer productivity and faster deployments

- Create shared infrastructure to help manage email subscriptions and unsubscribes across 5 different product teams

- Triage and respond to bug bounty reports

- Work with outside security auditors to run a penetration test against our applications

**Key Responsibilities:**


- Provide a stable modern platform for our applications to be deployed to

- Keep our applications and data secure

- Improve our developer experience and tools so the YC Software Team can ship product updates quickly

- Help debug performance issues in our web applications

- Ship Ruby on Rails and TypeScript code that is shared among multiple YC applications

**Skills:**


- You know how to develop and debug MVC web applications. We use Ruby on Rails, Postgres, and React, but don’t worry if you haven’t used those specific technologies before

- You know how to deploy and run applications with AWS and Docker

- You use Terraform to make infrastructure changes

**Experience:**


- You have at least 5 years of experience developing and deploying web applications

- You are deeply familiar with the AWS platform specifically ECS, RDS, IAM, and CloudWatch

**Location:** YC is headquartered in the SF Bay Area with employees working from home or from our offices in San Francisco and Mountain View, CA. Candidates must live in the SF Bay Area or be willing to relocate.

**Compensation:** $130,000 to $270,000 + bonus (depending on skills and experience).

**Benefits:** Our full benefits package includes medical, vision, and dental plans, infertility benefit, STD/LTD, life insurance, commuter benefits, flexible spending account, health savings account, 401(k) + 4% matching, generous parental leave, paid holidays and flexible paid time off policy.

**Work Authorization:** This position does not support work authorization/visa sponsorship.

Y Combinator considers qualified applicants with criminal histories, consistent with applicable federal, state, and local law including San Francisco’s Fair Chance Ordinance. Y Combinator is committed to protecting the privacy of the personal information of job applicants and complying with the California Consumer Privacy Act. The privacy policy of Ashby, Inc., the hiring platform used by Y Combinator, governs the collection of such data.

## Competency 1. **Cloud Infrastructure Management**



Candidates should demonstrate a strong understanding of AWS services, particularly ECS, RDS, IAM, and CloudWatch. They should be able to articulate how they have managed and evolved cloud infrastructure in past roles, ensuring stability, scalability, and security.

### Lead Question



**&ldquo;Describe your experience with managing AWS infrastructure, specifically ECS, RDS, IAM, and CloudWatch. Can you provide an example of a complex infrastructure challenge you faced and how you resolved it?&rdquo;** Listen for details on the candidate's approach to planning, problem-solving, and their understanding of AWS services. Probe for examples of when they had to troubleshoot or optimize cloud infrastructure.

&nbsp;

&nbsp;

_What to look for: Depth of knowledge in AWS services, problem-solving skills, and examples of past infrastructure challenges and solutions._

### Situational Scenario



**&ldquo;Imagine you are tasked with migrating a production application from EC2 to ECS. What steps would you take to ensure a smooth transition, and how would you handle potential issues that might arise during the migration?&rdquo;**

&nbsp;

&nbsp;

_What to look for: A structured approach to migration, risk assessment, and mitigation strategies._

### Diving Deeper




- &ldquo;How do you monitor and optimize AWS resource usage to manage costs effectively?&rdquo;

- &ldquo;Can you explain a time when you had to implement a disaster recovery plan for AWS infrastructure?&rdquo;

- &ldquo;What strategies do you use to ensure high availability and fault tolerance in the cloud?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Limited understanding of AWS services | Basic knowledge but lacks depth | Good understanding with some practical experience | Strong practical experience and problem-solving skills | Expert in AWS infrastructure management with a track record of complex problem-solving |

## Competency 2. **Web Application Development**



The candidate should be proficient in developing and debugging MVC web applications. While specific experience with Ruby on Rails, Postgres, and React is preferred, the ability to adapt to these technologies is also valuable.

### Lead Question



**&ldquo;Walk me through your process of developing and debugging an MVC web application. How do you ensure code quality and maintainability?&rdquo;** Look for a clear understanding of MVC architecture and best practices in coding and debugging. Ask for examples of how they have improved application performance or resolved complex bugs.

&nbsp;

&nbsp;

_What to look for: Proficiency in MVC web application development, debugging skills, and a commitment to code quality._

### Situational Scenario



**&ldquo;You've inherited a legacy Ruby on Rails application that is experiencing performance issues due to n+1 queries. How would you identify and resolve these issues?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Ability to diagnose performance issues, knowledge of Active Record, and effective problem-solving._

### Diving Deeper




- &ldquo;Describe a time when you refactored a significant portion of a web application. What was the outcome?&rdquo;

- &ldquo;How do you approach testing in web application development?&rdquo;

- &ldquo;Can you discuss a feature you implemented that significantly improved user experience or performance?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Struggles with MVC concepts and debugging | Basic development skills but limited debugging experience | Competent in development and debugging with some performance optimization | Strong development and debugging skills with a focus on maintainability and performance | Expert in web application development with a portfolio of optimized and high-quality applications |

## Competency 3. **Deployment and Operations**



Candidates should have a solid understanding of deploying and running applications in a cloud environment, using tools like AWS and Docker. They should also be familiar with using Terraform for infrastructure as code.

### Lead Question



**&ldquo;Describe your experience with deploying web applications using AWS and Docker. How do you ensure deployments are reliable and efficient?&rdquo;** Listen for specific examples of deployment strategies, use of Docker for containerization, and how they manage deployment pipelines.

&nbsp;

&nbsp;

_What to look for: Knowledge of deployment processes, experience with Docker, and the ability to create efficient deployment pipelines._

### Situational Scenario



**&ldquo;You need to roll out an update to a critical application during peak usage hours. How would you plan and execute the deployment to minimize downtime and risk?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Risk assessment, planning skills, and strategies for zero-downtime deployments._

### Diving Deeper




- &ldquo;How do you use Terraform in your current role? Can you give an example of an infrastructure change you implemented with it?&rdquo;

- &ldquo;What is your approach to managing application secrets in a cloud environment?&rdquo;

- &ldquo;How do you ensure that your deployment process is repeatable and scalable?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Limited experience with cloud deployments | Basic deployment skills with some Docker usage | Good deployment skills with experience in AWS and containerization | Strong experience with deployment pipelines and infrastructure as code | Expert in cloud deployments with a track record of efficient and reliable operations |

## Competency 4. **Security and Performance**



Candidates should be able to demonstrate their ability to keep applications and data secure, as well as their experience in debugging and improving the performance of web applications.

### Lead Question



**&ldquo;How do you approach securing web applications and protecting sensitive data? Can you share an example of a security or performance challenge you've addressed?&rdquo;** Look for a comprehensive approach to security, including knowledge of common vulnerabilities and performance bottlenecks.

&nbsp;

&nbsp;

_What to look for: A strong security mindset, experience with performance optimization, and examples of past challenges._

### Situational Scenario



**&ldquo;A penetration test has revealed several security vulnerabilities in an application you're responsible for. How would you prioritize and address these issues?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Prioritization skills, a systematic approach to resolving security issues, and knowledge of best practices._

### Diving Deeper




- &ldquo;Describe your experience with implementing CSRF protection in a web application.&rdquo;

- &ldquo;How do you ensure that your applications are resilient against common security threats?&rdquo;

- &ldquo;What tools and techniques do you use to monitor and improve the performance of web applications?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Limited understanding of web application security | Basic knowledge of security with some experience in performance tuning | Good grasp of security principles with a focus on application performance | Strong experience in securing applications and optimizing performance | Expert in web application security and performance with a proven track record of resolving complex issues |

## Competency 5. **Collaboration and Communication**



Candidates should exhibit strong communication skills and the ability to collaborate with product engineers and external parties, such as security auditors, reflecting Y Combinator's values of respect and accountability.

### Lead Question



**&ldquo;Can you describe a time when you had to collaborate with a team to solve an infrastructure issue? How did you communicate your ideas and ensure everyone was on the same page?&rdquo;** Look for examples of effective communication, teamwork, and the ability to work towards a common goal.

&nbsp;

&nbsp;

_What to look for: Communication skills, teamwork, and the ability to foster collaboration._

### Situational Scenario



**&ldquo;You're working with a product team that is unfamiliar with the infrastructure changes you're implementing. How would you explain these changes to them and ensure their buy-in?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Ability to explain technical concepts to non-technical stakeholders and to build consensus._

### Diving Deeper




- &ldquo;How do you handle disagreements or conflicts within a team regarding technical decisions?&rdquo;

- &ldquo;Can you give an example of how you've mentored or supported a less experienced team member?&rdquo;

- &ldquo;Describe a situation where you had to communicate complex technical information to a non-technical audience.&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Struggles with clear communication and teamwork | Communicates adequately but lacks collaborative skills | Good communicator with some collaborative successes | Strong team player with effective communication and mentoring skills | Exceptional communicator and collaborator with a track record of team leadership |

## Competency 6. **Problem-Solving and Agility**



Candidates should demonstrate problem-solving skills and the ability to respond quickly to changes, aligning with Y Combinator's emphasis on a founder-first and agile approach.

### Lead Question



**&ldquo;Tell me about a time when you had to quickly adapt to a significant change in a project. How did you approach the problem and what was the outcome?&rdquo;** Seek examples of agility in thought and action, as well as the ability to find effective solutions under pressure.

&nbsp;

&nbsp;

_What to look for: Problem-solving skills, adaptability, and examples of quick thinking._

### Situational Scenario



**&ldquo;You discover a critical bug in a live application that affects user data. You have limited time to fix it before it impacts more users. What steps do you take?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Quick assessment of the situation, prioritization of actions, and effective resolution._

### Diving Deeper




- &ldquo;How do you stay updated with the latest industry trends and technologies?&rdquo;

- &ldquo;Describe a situation where you had to learn a new technology or framework quickly tosolve a problem.&rdquo;

- &ldquo;Can you talk about a time when you had to pivot your strategy due to market changes or feedback?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Struggles with problem-solving and resistance to change | Can solve basic problems but is slow to adapt | Good problem-solver with some adaptability | Strong problem-solving skills and is fairly adaptable to change | Exceptional problem-solver with a proven ability to adapt quickly and innovate |

## Case Study



Y Combinator is experiencing slow deployment times for its suite of web applications, which is affecting the ability to roll out updates quickly and efficiently. The current deployment process is manual and prone to errors, leading to occasional downtime and frustrated developers.

You are provided with the current deployment scripts, logs of recent deployment issues, and feedback from the product engineering team. Your task is to analyze the existing process, identify bottlenecks, and propose a new deployment strategy.

Questions to address:


- What are the root causes of the slow deployment times?

- How would you redesign the deployment process to improve speed and reliability?

- What risks are associated with your proposed strategy, and how would you mitigate them?

- Which metrics would you use to measure the success of the new deployment process?

A successful solution will significantly reduce deployment times, minimize downtime, and improve the overall developer experience. It should also be scalable, cost-effective, and align with Y Combinator's values of efficiency and responsiveness.

Your proposal should consider budget constraints, the need for minimal disruption during the transition, and the diverse technical backgrounds of the product engineering team.

Evaluation criteria will include the thoroughness of the analysis, the feasibility and creativity of the proposed solution, and the clarity of the presentation.

## Candidate Q&A



The Candidate Q&A is an opportunity for you to ask questions about the role, Y Combinator's culture, and anything else you'd like to know. It's also a chance for us to understand your thought process, problem-solving approach, and how you align with our values.

**Q. What interests you about working at Y Combinator?**

A. I'm passionate about the startup ecosystem and the opportunity to work with innovative teams. Y Combinator's mission to increase the number of successful startups is inspiring, and I believe my skills in infrastructure management can contribute to that goal.

**Q. How do you approach learning new technologies?**

A. I stay curious and proactive. I follow industry news, participate in online communities, and take on personal projects to experiment with new tools and frameworks.

**Q. Can you describe a challenging project you worked on and how you overcame obstacles?**

A. I once led a project to migrate a legacy system to the cloud. We faced several technical and team alignment challenges. I facilitated open communication, broke down the project into manageable parts, and ensured we had a clear roadmap, which led to a successful migration.

**Q. How do you balance technical decisions with business needs?**

A. I prioritize understanding the business objectives behind technical decisions. I weigh the pros and cons, consider the impact on the product and users, and aim for solutions that meet both technical and business requirements.

**Q. What do you think is the most important trend in infrastructure technology today?**

A. The shift towards serverless architectures is significant. It offers scalability, cost savings, and allows engineers to focus on building features rather than managing servers.

**Q. How do you handle stress and tight deadlines?**

A. I stay organized, prioritize tasks based on impact, and communicate clearly with my team. I also ensure to take breaks and maintain a healthy work-life balance to stay sharp and focused.

## Candidate Evaluation

Complete the following scorecard for the Infrastructure Software Engineer role at Y Combinator. Refer to the evaluation guides for each competency to determine the candidate's score.

| Competency | Rating (1-5) | Interviewer Comments |
| --- | --- | --- |
| Cloud Infrastructure Management |   |   |
| Web Application Development |   |   |
| Deployment and Operations |   |   |
| Security and Performance |   |   |
| Collaboration and Communication |   |   |
| Problem-Solving and Agility |   |   |
| &nbsp; | | |
| **Total** |   |   |

